<a href="https://colab.research.google.com/github/zahraa-m/PHE_UK/blob/Zahraa/PHE_UK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In the code below, I prepared the data to:
### 1.   Produce a cross table to calculate the total quantity of each medicine per area in Greater Manchester.
### 2.   Produce a cross table to calculate the cost of each medicine per area in Greater Manchester.
### 3.   Calculate the difference in the total quantity of each medicine between 2020 and 2021.
### 4.   Calculate the increased percentage of the total quantity for each medicine.
### 5.   Calculate the difference in cost of each medicine between 2020 and 2021.
### 6.   Calculate the increased percentage of the cost for each medicine.



In [2]:
import pandas as pd
df_2020= pd.read_csv('https://raw.githubusercontent.com/zahraa-m/PHE_UK/main/EPD_202004.csv')
df_2021= pd.read_csv('https://raw.githubusercontent.com/zahraa-m/PHE_UK/main/EPD_202104.csv')

df_2020['area_co'] = df_2020['POSTCODE'].str[0:3]
df_2021['area_co'] = df_2021['POSTCODE'].str[0:3]

#calculate total quantity per medicine in each area in the Greater Manchester
TQ_per_med_2020 = pd.crosstab(index=df_2020['CHEMICAL_SUBSTANCE_BNF_DESCR'], columns=df_2020['area_co'], values=df_2020['TOTAL_QUANTITY'], aggfunc='sum', margins=True).fillna(0).astype(int)
TQ_per_med_2021 = pd.crosstab(index=df_2021['CHEMICAL_SUBSTANCE_BNF_DESCR'], columns=df_2021['area_co'], values=df_2021['TOTAL_QUANTITY'], aggfunc='sum', margins=True).fillna(0).astype(int)

TQ_per_med_2020.to_csv("TQ_2020.csv")
TQ_per_med_2021.to_csv("TQ_2021.csv")


#calculate Cost per medicine in each area in the Greater Manchester
cost_per_med_2020 = pd.crosstab(index=df_2020['CHEMICAL_SUBSTANCE_BNF_DESCR'], columns=df_2020['area_co'], values=df_2020['ACTUAL_COST'], aggfunc='sum', margins=True).fillna(0).astype(int)
cost_per_med_2021 = pd.crosstab(index=df_2021['CHEMICAL_SUBSTANCE_BNF_DESCR'], columns=df_2021['area_co'], values=df_2021['ACTUAL_COST'], aggfunc='sum', margins=True).fillna(0).astype(int)

#Delete the "All" row in the table
TQ_per_med_2020 = TQ_per_med_2020.drop(["All"], axis=0)
TQ_per_med_2021 = TQ_per_med_2021.drop(["All"], axis=0)

cost_per_med_2020 = cost_per_med_2020.drop(["All"], axis=0)
cost_per_med_2021 = cost_per_med_2021.drop(["All"], axis=0)

#Rename the All column to TQ_2020 and TQ_2021
TQ_per_med_2020 = TQ_per_med_2020.rename(columns={"All": "TQ_2020"})
TQ_per_med_2021 = TQ_per_med_2021.rename(columns={"All": "TQ_2021"})

cost_per_med_2020 = cost_per_med_2020.rename(columns={"All": "cost_2020"})
cost_per_med_2021 = cost_per_med_2021.rename(columns={"All": "cost_2021"})

#filter the tables to include only medicine's name and total quantity
df1 = TQ_per_med_2020.filter(['CHEMICAL_SUBSTANCE_BNF_DESCR', 'TQ_2020'])
df2 = TQ_per_med_2021.filter(['CHEMICAL_SUBSTANCE_BNF_DESCR', 'TQ_2021'])

#filter the tables to include only medicine's name and Cost
df3 = cost_per_med_2020.filter(['CHEMICAL_SUBSTANCE_BNF_DESCR', 'cost_2020'])
df4 = cost_per_med_2021.filter(['CHEMICAL_SUBSTANCE_BNF_DESCR', 'cost_2021'])

#find the difference between Total quantity of 2020 and 2021
TQ_per_drug_allyear = pd.merge(df1, df2, how="outer", on=('CHEMICAL_SUBSTANCE_BNF_DESCR')).fillna(0).astype(int)
diff_TQ = (df2['TQ_2021'] - df1['TQ_2020'] )
TQ_per_drug_allyear['diff_TQ']=diff_TQ.fillna(0) 

#find the percentage of the difference
diff_TQ_pct = (diff_TQ / df1['TQ_2020']) * 100
TQ_per_drug_allyear['diff_TQ_pct']=diff_TQ_pct.fillna(0)

#Filter zero differance in total quantity
TQ_per_drug_allyear= TQ_per_drug_allyear.query("diff_TQ>0")


#find the difference between cost of medicine in 2020 and 2021
cost_per_drug_allyear = pd.merge(df3, df4, how="outer", on=('CHEMICAL_SUBSTANCE_BNF_DESCR')).fillna(0).astype(int)
diff_cost = (df4['cost_2021'] - df3['cost_2020'])
cost_per_drug_allyear['diff_cost']= diff_cost.fillna(0)

#find the percentage of the difference
diff_cost_pct = (diff_cost / df3['cost_2020']) * 100
cost_per_drug_allyear['diff_cost_pct']= diff_cost_pct.fillna(0)

#Filter zero differance in cost
cost_per_drug_allyear=cost_per_drug_allyear.query("diff_cost>0")






## Below I plotted the data that I have prepared above. You would need to install **Plotly** package to see the plots. Plotly is used to create interactive plots and maps. Please use this code to install the package:


In [6]:
!pip install -U plotly

     |████████████████████████████████| 26.5 MB 1.7 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1



## After installing Plotly, we can run the next code to illustrate the PHE data.


In [3]:

#plot the difference between Total quantity of medicine in 2021 and 2020
import plotly.express as px
fig_TQ = px.line(TQ_per_drug_allyear, x=TQ_per_drug_allyear.index, y='diff_TQ',
                  labels={
                     "x": "Medicines",
                     "diff_TQ": "Total Quantity difference"},
                 title='Total Quantity difference between 2021 & 2020')
fig_TQ.show()

#plot the percentage of difference between Total quantity of medicine in 2021 and 2020
fig_TQ_pct = px.line(TQ_per_drug_allyear, x=TQ_per_drug_allyear.index, y='diff_TQ_pct',
                  labels={
                     "x": "Medicines",
                     "diff_TQ_pct": "Total Quantity Increase %"},
                 title='Percentage increase of Total Quantity difference between 2021 & 2020')
fig_TQ_pct.update_traces(line_color='#F54730')
fig_TQ_pct.show()


#plot the cost difference between the medicine of 2021 and 2020
fig_cost = px.line(cost_per_drug_allyear, x=cost_per_drug_allyear.index, y='diff_cost',
                    labels={
                       "x": "Medicines",
                       "diff_cost": "Cost difference"},
                   title='Cost difference between 2021 & 2020')
fig_cost.show()

#plot the percentage of cost difference between the medicine of 2021 and 2020
fig_cost_pct = px.line(cost_per_drug_allyear, x=cost_per_drug_allyear.index, y='diff_cost_pct',
                  labels={
                     "x": "Medicines",
                     "diff_cost_pct": "Cost Increase %"},
                 title='Percentage increase of Cost difference between 2021 & 2020')
fig_cost_pct.update_traces(line_color='#F54730')
fig_cost_pct.show()





## Show the table of total quantity of medicine in 2021 sorted by the increase precentage



In [6]:
import plotly.graph_objects as go


TQ_per_drug_allyear= TQ_per_drug_allyear.sort_values('diff_TQ_pct', ascending=False)
TQ_per_drug_allyear = TQ_per_drug_allyear.reset_index(drop=False, inplace=False)
fig_tab = go.Figure(data=[go.Table(
    header=dict(values=['Medicine Name', 'Total Quantity 2020', 'Total Quantity 2021', 'Diff between 2020 & 2021', 'Increase Percentage PCT'],
                align='left'),
    cells=dict(values=[TQ_per_drug_allyear.CHEMICAL_SUBSTANCE_BNF_DESCR, TQ_per_drug_allyear.TQ_2020, TQ_per_drug_allyear.TQ_2021, TQ_per_drug_allyear.diff_TQ, TQ_per_drug_allyear.diff_TQ_pct],
               align='left'))
])

fig_tab.show()


## Plot Manchester map to show the distribution of the medicine "Influenza" in 2021.


In [7]:
import plotly.express as px1
from urllib.request import urlopen
import json

with urlopen('https://raw.githubusercontent.com/zahraa-m/PHE_UK/main/manchester.geojson') as response:
  manches = json.load(response)


map1 = TQ_per_med_2021.loc[TQ_per_med_2021.index == 'Influenza'].T
map1 = map1.reset_index(drop=False, inplace=False)
map1.drop(map1.tail(1).index,inplace=True) 

fig_map1 = px1.choropleth(map1, geojson=manches, color="Influenza",
                        locations="area_co", featureidkey="properties.area_co", 
                        scope="europe", projection="mercator",
                        color_continuous_scale="Blues", range_color=(0, 300))
fig_map1.update_geos(fitbounds="locations", visible=False)
fig_map1.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig_map1.update_geos(fitbounds="locations", visible=False)
fig_map1.show()